<a href="https://colab.research.google.com/github/mozzaquatro/financial_market/blob/main/08_Analise_Fundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

2. Obtendo e tratando os dados

In [2]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [4]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [5]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PMET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"
1,POPR4,10.17,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,5.458030e+08,0.82,"30,93%"
2,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
3,CSTB4,147.69,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
4,CFLU4,1000.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"8,88%","10,72%",1.10,"17,68%","32,15%",0.0,6.035100e+07,0.06,"8,14%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
978,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
979,GLOB4,30.49,4873.60,8.59,1.512,"0,00%",1.389,-63.21,46.91,-4.59,55.03,28.77,"3,22%","0,03%",0.96,"3,90%","0,18%",0.0,5.675000e+09,1.76,"5,35%"
980,LWSA3,19.10,26684.90,3.82,16.215,"0,14%",2.680,6.67,357.41,10.90,303.59,87.80,"4,54%","0,06%",3.82,"1,33%","0,01%",140686000.0,2.950560e+09,0.04,"53,23%"


In [6]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float')/100

In [7]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PMET3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0410,0.0,-2.908630e+08,0.00,0.3774
1,POPR4,10.17,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0866,0.0565,1.08,0.1525,0.1993,0.0,5.458030e+08,0.82,0.3093
2,MNSA3,0.42,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,-2.0815,-3.6266,3.63,-0.1350,1.4570,0.0,-9.105000e+06,-6.52,-0.4111
3,CSTB4,147.69,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.0,8.420670e+09,0.14,0.3191
4,CFLU4,1000.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0888,0.1072,1.10,0.1768,0.3215,0.0,6.035100e+07,0.06,0.0814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,UBBR11,14.75,1201.81,3.91,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
978,UBBR3,18.00,1466.61,4.77,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
979,GLOB4,30.49,4873.60,8.59,1.512,0.0000,1.389,-63.21,46.91,-4.59,55.03,28.77,0.0322,0.0003,0.96,0.0390,0.0018,0.0,5.675000e+09,1.76,0.0535
980,LWSA3,19.10,26684.90,3.82,16.215,0.0014,2.680,6.67,357.41,10.90,303.59,87.80,0.0454,0.0006,3.82,0.0133,0.0001,140686000.0,2.950560e+09,0.04,0.5323


**Analisando os dados**

empresa com liquidez 
1.000.000 média diária dos ultimos 2 meses

In [8]:
df = df[df['Liq.2meses'] > 1000000]

menor relacao EV/Ebit e
maior ROI

In [9]:
ranking = pd.DataFrame()
ranking['pos']=range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [10]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,CMIN3
1,2,GPIV33,WIZS3
2,3,GOAU3,PSSA3
3,4,GOAU4,TASA4
4,5,USIM3,TASA3
...,...,...,...
145,146,VIIA3,LAME3
146,147,ALSO3,LAME4
147,148,ENEV3,VLID3
148,149,LREN3,MULT3


In [11]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [12]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [13]:
t = pd.concat([a,b])
t

,AALR3,ABEV3,ALLD3,ALSO3,ALUP11,AMAR3,ASAI3,AURA33,B3SA3,BEEF3,BLAU3,BMOB3,BRFS3,BRKM5,BRML3,CAML3,CARD3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,COCE5,CPFE3,CPLE11,CPLE3,CPLE6,CRFB3,CRPG5,CSED3,CSMG3,CSNA3,CURY3,CYRE3,DEXP3,DIRR3,DMMO3,DXCO3,ECOR3,...,TASA3,TASA4,TECN3,TEND3,TGMA3,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP3,UNIP6,USIM3,USIM5,VALE3,VBBR3,VIIA3,VITT3,VIVT3,VLID3,VULC3,WIZS3,AERI3,ALPA4,AMBP3,ARZZ3,BOAS3,INTB3,MOSI3,MULT3,ORVR3,PETZ3,RADL3,RDOR3,RRRP3,TFCO4,VAMO3,VIVA3,WEGE3
pos,144.0,139,26,147.0,21,96,116,76.0,102,51,121,87,115,10,137.0,86,81,112,80,15,75,59,11,106,66,48,45,49,88,94,97,54,9,47,98,61,55,22,91,73,...,28,27,33,67,84,82,141,72,38,74,133,37,36,5,6,8,100,146.0,126,136.0,107,140,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,NaN,62,46,NaN,40,118,57,NaN,37,36,14,48,127,15,NaN,84,47,110,133,126,108,107,1,143,86,124,123,125,98,28,136,101,11,34,137,55,141,12,69,79,...,5,4,83,121,59,120,49,117,63,91,139,9,8,17,16,6,80,NaN,43,NaN,148,129,2,114.0,39.0,144.0,96.0,95.0,56.0,38.0,149.0,150.0,105.0,99.0,111.0,112.0,25.0,82.0,53.0,30.0


In [14]:
rank = t.dropna(axis=1).sum()
rank

ABEV3     201
ALLD3      72
ALUP11     61
AMAR3     214
ASAI3     173
         ... 
VBBR3     180
VITT3     169
VLID3     255
VULC3     269
WIZS3      18
Length: 133, dtype: int64

In [15]:
rank.sort_values()[:15]

PSSA3     4
CMIN3    12
VALE3    14
WIZS3    18
CSNA3    20
ETER3    21
USIM5    22
USIM3    22
BRKM5    25
GOAU3    26
GOAU4    26
MRFG3    29
TASA4    31
TASA3    33
GGBR3    34
dtype: int64